In [1]:
import numpy as np
from nltk.corpus import stopwords
from tqdm import tqdm
from tqdm import trange
import jsonlines
from scipy import sparse
from os import listdir
import itertools

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
global_boy = 0

def error_safe_gen(gen):
    global global_boy
    while True:
        try:
            yield next(gen)
        except StopIteration:
            raise
        except Exception as e:
            global_boy += 1
            pass

vectors = {}
stopwords_ru = stopwords.words('russian')
dimentions = 0
with open('wiki.ru.vec', 'r', encoding='utf-8') as f_vectors:
    file_stats = f_vectors.readline().split()
    print(file_stats)
    length = int(file_stats[0])
    dimentions = int(file_stats[1])
    for line in tqdm(error_safe_gen(f_vectors), total=length):
        line = line.split()
        if line[0] not in stopwords_ru:
            vectors[line[0]] = np.array([float(l) for l in line[len(line) - dimentions:]])
  
    print(global_boy)

['1888423', '300']


100%|██████████████████████████████████████████████████████████████████████| 1888423/1888423 [06:20<00:00, 4958.08it/s]


0


In [3]:
path = listdir('c')
path = sorted(path)[:9]
#path = ['AA', 'AD', 'AE']
print(path)

data = []
for p in tqdm(path):
    with jsonlines.open('c/' + p, 'r') as f:
        for entry in f:
            data.append(entry)

['AA', 'AB', 'AD', 'AE', 'AF', 'AI', 'AJ', 'AK', 'AQ']


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:58<00:00, 13.12s/it]


In [4]:
categories = [d['categories'] for d in data]
categories = sorted(set(list(itertools.chain(*categories))))

categories_dict = {c: 0 for c in categories}
for d in data:
    for c in d['categories']:
        if categories_dict.get(c) is not None:
            categories_dict[c] += 1
            
ok = [c for c in categories if categories_dict[c] > 5 and categories_dict[c] < 5000]
ok_set = set(ok)
categories_dict = {d['id']: [c for c in d['categories'] if c in ok_set] for d in tqdm(data)}
len(ok)

100%|██████████████████████████████████████████████████████████████████████| 116584/116584 [00:00<00:00, 210635.18it/s]


14301

In [5]:
text_vectors = {}
skipped = []
for d in tqdm(data):
    if len(set(d['categories']).intersection(ok_set)) > 0:
        text_vec = np.zeros(dimentions)
        id = d['id']
        text = d['text']
        words = text.split()
        words = [w for w in words if w != '.' and w != ',']
        n = 0
        for w in words:
            vec = vectors.get(w)
            if vec is not None:
                if len(vec) != 300:
                    print(w)
                    print(vec)
                text_vec += vec
                n += 1
        if n == 0:
            skipped.append(id)
            continue
        text_vec = text_vec / n
        text_vectors[id] = text_vec
    else:
        skipped.append(d['id'])

print(len(skipped))

100%|████████████████████████████████████████████████████████████████████████| 116584/116584 [01:39<00:00, 1165.87it/s]


22638


In [6]:
ids = [id for id in [d['id'] for d in data] if id in text_vectors.keys()]
print(- len(ids) + len(categories_dict))

Y = sparse.dok_matrix((len(ids), len(ok)), dtype=np.int)
for i in trange(len(ids)):
    for c in categories_dict[ids[i]]:
        Y[i, ok.index(c)] = 1

X = np.array([text_vectors[id] for id in ids])

22638


100%|██████████████████████████████████████████████████████████████████████████| 93946/93946 [01:29<00:00, 1044.93it/s]


In [ ]:
kneighbours = KNeighborsClassifier(n_neighbors=8, n_jobs=-1)
Y_csr = Y.tocsr()

kf = KFold(n=len(ids), n_folds=5, shuffle=True)


train_index, test_index = 0, 0
for item in kf:
    train_index, test_index = item[0], item[1]

print("TRAIN:", train_index, "TEST:", test_index)
print(len(train_index))
print(len(test_index))
X_train = X[train_index]
Y_train = Y_csr[train_index]
print('Ok')
kneighbours.fit(X_train, Y_train)

TRAIN: [    1     3     4 ..., 93943 93944 93945] TEST: [    0     2    14 ..., 93921 93923 93929]
75157
18789
Ok


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=8, p=2,
           weights='uniform')

In [ ]:
precision = 0
recall = 0

splits = int(len(test_index) / 100)
for i in trange(splits):
    X_test = X[test_index[i * 100 : (i+1) * 100]]
    Y_test = Y[test_index[i * 100 : (i+1) * 100]]
    res = kneighbours.predict(X_test)
    precision += precision_score(Y_test, res)
    recall += recall_score(Y_test, res)
    
print(precision / splits)
print(recall / splits)

  0%|                                                                                          | 0/187 [00:00<?, ?it/s]

# Черновик

In [3]:
text_vectors = {}
with jsonlines.open('texts.jl', 'r') as file:
    for text in tqdm(file):
        text_vec = np.zeros(dimentions)
        id = text['id']
        text = text['text']
        words = text.split()
        words = [w for w in words if w != '.' and w != ',']
        n = 0
        for w in words:
            vec = vectors.get(w)
            if vec is not None:
                if len(vec) != 300:
                    print(w)
                    print(vec)
                text_vec += vec
                n += 1
        text_vec = text_vec / n
        text_vectors[id] = text_vec

5107it [00:26, 189.95it/s]C:\Users\Maxim\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in true_divide
116555it [03:55, 494.00it/s]


In [7]:
kneighbours = KNeighborsClassifier(n_neighbors=8)

print(X.shape)
print(Y.shape)
i = 0
for j in range(14301):
    i += Y[10, j]

print(i)
cross_val_score(kneighbours, X, Y)

NameError: name 'OneVsRestClassifier' is not defined

In [58]:
kneighbours = KNeighborsClassifier(n_neighbors=8, n_jobs=-1)

kf = KFold(n=len(ids), n_folds=5, shuffle=False)
for train_index, test_index in kf:
    print("TRAIN:", train_index, "TEST:", test_index)
    print(len(train_index))
    print(len(test_index))
    X_train = X[train_index]
    Y_train = Y[train_index[0]:train_index[-1]+1]
    print(X_train.shape)
    print(Y_train.shape)
    kneighbours.fit(X_train, Y_train)
    
    for i in trange(int(len(test_index) / 100)):
        X_test = X[test_index[i * 100]:test_index[(i+1) * 100]]
        Y_test = Y[test_index[i * 100]:test_index[(i+1) * 100]]
        res = kneighbours.predict(X_test)
    break

TRAIN: [18790 18791 18792 ..., 93943 93944 93945] TEST: [    0     1     2 ..., 18787 18788 18789]
75156
18790
(75156, 300)
(75156, 14301)


 12%|█████████▉                                                                       | 23/187 [01:12<08:34,  3.14s/it]

KeyboardInterrupt: 

In [20]:
print(next((d for d in data if d['id'] == skipped[0]), None))

{'categories': ['860 год'], 'text': '860 год\n\n\n"См. также: "\n\n"См. также: "\n', 'title': '860 год', 'id': '6056'}


In [39]:
X_kek = np.array([5, 6, 7])
Y_kek = sparse.dok_matrix((3, 2), dtype=int)
Y_kek[0, 0] = 1
Y_kek[1, 1] = 2
Y_kek[0, 1] = 3
Y_kek[:2].todense()

matrix([[1, 3],
        [0, 2]])

In [37]:
kneighbours = KNeighborsClassifier(n_neighbors=8, n_jobs=-1)

kf = KFold(n=3, n_folds=3, shuffle=True)
for train_index, test_index in kf:
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_kek[train_index], X_kek[test_index]
    print(X_train, X_test)
    y_train, y_test = Y_kek[train_index], Y_kek[test_index]
    print(y_train.todense(), y_test.todense())

TRAIN: [0 2] TEST: [1]
[5 7] [6]
[[1 3]
 [0 0]] [[0 2]]
TRAIN: [1 2] TEST: [0]
[6 7] [5]
[[0 2]
 [0 0]] [[1 3]]
TRAIN: [0 1] TEST: [2]
[5 6] [7]
[[1 3]
 [0 2]] [[0 0]]
